In [15]:
import mesa
import numpy as np
import mesa.time as mt
import mesa.space as ms
import mesa.batchrunner as mb
import matplotlib.pyplot as plt
import rasterio
from PIL import Image
import math


In [16]:
# Read the tif file with DEM data
dem_file = "data/UK_DEM.tif"
dem_image = Image.open(dem_file)

# store the dem data into array
dem_array = np.array(dem_image)

In [17]:
# Obtain geographic resolution information
with rasterio.open(dem_file) as src:
    x_resolution = src.transform[0]
    y_resolution = src.transform[4]
    # The average of the x and y directions is taken as the geographic resolution of the pixel
    pixel_resolution = (abs(x_resolution) + abs(y_resolution)) / 2.0
    print("Each pixel of the DEM equals [" + str(pixel_resolution) + "m] in the real world")

Each pixel of the DEM equals [10.0m] in the real world


In [18]:
# Variable definition
T = 25    # Average temperature
h = 0.6   # Relative humidity
dt = 0.0000001    # Time cost in each step

# Variable will change when model runs
s = pixel_resolution    # Length of the patch side in real world
h_i = 0   # Elevation of patch i
h_i1 = 0  # Elevation of patch i+1

In [19]:
# Constant definition
a = 0.03
c = 0.05
D = -0.3

In [20]:
# Definition of spread rate
R_0 = a*T + c*h - D   # Flat, windless, homogeneous burning speed

# Constant definition 
K_s = 1  # Combustibles coefficient, which indicates the effect of combustibles on the progress of combustion.
K_w = 1  # The wind direction coefficient, which indicates the effect of windiness and wind direction on the progress of combustion. In this paper the effect of wind is not considered.



K_phi = 1 # Slope factor, which indicates the effect of elevation difference on the burning progress.
K_phi_4 = math.exp(3.533 * pow( (abs(h_i-h_i1) / s ), 1.2))                      # Slope factor, calculation formula in the four-neighborhood domain
K_phi_8 = math.exp(3.533 * pow( (abs(h_i-h_i1) / (math.sqrt(2)*s)), 1.2))        # Slope factor, calculation formula in the eight-neighborhood diagonal

# Definition of integrated propagation speed
R = R_0 * K_s * K_w * K_phi
R_4 = R_0 * K_s * K_w * K_phi_4   # spread in the four-neighborhood domain
R_8 = R_0 * K_s * K_w * K_phi_8   # spread in the eight-neighborhood diagonal


#### Definition of the state update

$$State_{i,j}^{t+1} = State_{i,j}^{t} + \frac{(R_{i+1,j}^{t}+R_{i-1,j}^{t}+R_{i,j+1}^{t}+R_{i,j-1}^{t})\cdot dt}{s} + \frac{({R_{i-1,j-1}^t}^2 +{R_{i-1,j+1}^t}^2 + {R_{i+1,j-1}^t}^2 + {R_{i+1,j+1}^t}^2)\cdot dt}{2\cdot s^2} $$

In [21]:
# defined the each patch
class Patch:
    def __init__(self, pos, grid):
        self.pos = pos
        self.grid = grid     # self.grid
        self.elevation = None  # elevation value of each patch
        self.state = 0         # fire or not    0 = no fire; 0-1 = on fire; 2 = total burning; 3 = extinguished

    # Definition of an agent's behavior
    # Check if it has been affected by its neighbour 
    def check_affected(self):
        flag = False
        for each in self.grid.get_neighbors(self.pos, moore=True):
            if each.state>0:
                flag = True
        return flag
    
    # Update the state value if affected
    def update_state(self):
        if self.check_affected():
            
            
            return

        return
    

    # get the single R, self as point1
    def get_R(self, x2, y2, mode = 4):
        elevation_1 = self.elevation
        if mode == 4:
            elevation_2 = self.grid[x2][y2].elevation
            R_4_value = R_0 * K_s * K_w * math.exp(3.533 * pow( (abs(elevation_1-elevation_2) / s ), 1.2))
            return R_4_value
        elif mode == 8:
            elevation_2 = self.grid[x2][y2].elevation
            R_8_value = R_0 * K_s * K_w * math.exp(3.533 * pow( (abs(elevation_1-elevation_2) / (math.sqrt(2)*s)), 1.2))
            return R_8_value
        else:
            print("get_R: Worng mode! please check the input")


    # get the result of sum R, self as point1
    def get_R_summary(self, mode = 4):
        x = self.pos[0]
        y = self.pos[1]

        if mode == 4:
            R_4_summary_value = (self.get_R(x+1, y, 4) + \
                                 self.get_R(x-1, y, 4) + \
                                 self.get_R(x, y+1, 4) + \
                                 self.get_R(x, y-1, 4))*dt/s
            return R_4_summary_value
        elif mode == 8:
            R_8_summary_value = (math.pow(self.get_R(x+1, y+1, 8), 2) + \
                                 math.pow(self.get_R(x+1, y-1, 8), 2) + \
                                 math.pow(self.get_R(x-1, y+1, 8), 2) + \
                                 math.pow(self.get_R(x-1, y-1, 8), 2)) * dt \
                                 / (2*math.pow(s,2))
            return R_8_summary_value
        else:
            print("get_R_summary: Worng mode! please check the input")




            
    



In [22]:
# defined the abm world and create the patch agents
class ABMWorld(ms.SingleGrid):
    def __init__(self, dem_array):
        self.height = dem_array.shape[0]
        self.width = dem_array.shape[1]
        super().__init__(self.width, self.height, False)


In [23]:
# Create the abm model
class ABMModel(mt.Model):
    def __init__(self, dem_array):
        self.schedule = mt.RandomActivation(self)
        self.grid = ABMWorld(dem_array)

        # set the elevation
        for x in range(self.grid.width):
            for y in range(self.grid.height):
                elevation = dem_array[y, x]
                patch = Patch((x, y), self.grid)
                patch.elevation = elevation
                self.grid.place_agent(patch, (x, y))


    # each step what will happen?
    def step(self):
        self.schedule.step()
        self.get_fire()

    # get the count of agents
    def get_agent_counts(self):
        print(self.schedule.get_agent_count())

    # set a fire at XY
    def set_fire(self, x, y):
        fire_patch = self.grid[x][y]
        fire_patch.state = 1
        print("Well done, you set up a fire at " + str(fire_patch.pos))

    # get the fire from its neigbhours
    def get_fire(self):
        for each_patch in self.grid:
            if self.check_affected(each_patch.pos):
                patch_x = each_patch.pos[0]
                patch_y = each_patch.pos[1]

                if patch_x not in (0, 1885, 1934) and patch_y not in (0, 1885, 1934):
                    try:
                        each_patch.state = each_patch.state + self.get_R_summary(patch_x, patch_y, 4) + self.get_R_summary(patch_x, patch_y, 8)
                    except:
                        continue
                    self.check_state(patch_x, patch_y)


    # Check if it has been affected by its neighbour     
    def check_affected(self, pos):
        flag = False
        for each in self.grid.get_neighbors(pos, moore=True):
            if each.state>0:
                flag = True

        return flag

    # get the single R
    def get_R(self, x1, y1, x2, y2, mode = 4):
        if mode == 4:
            elevation_1 = self.grid[x1][y1].elevation
            elevation_2 = self.grid[x2][y2].elevation
            R_4_value = R_0 * K_s * K_w * math.exp(3.533 * pow( (abs(elevation_1-elevation_2) / s ), 1.2))
            return R_4_value
        elif mode == 8:
            elevation_1 = self.grid[x1][y1].elevation
            elevation_2 = self.grid[x2][y2].elevation
            R_8_value = R_0 * K_s * K_w * math.exp(3.533 * pow( (abs(elevation_1-elevation_2) / (math.sqrt(2)*s)), 1.2))
            return R_8_value
        else:
            print("get_R: Worng mode! please check the input")
    
    # get the result of sum R
    def get_R_summary(self, x, y, mode = 4):
        if mode == 4:
            R_4_summary_value = (self.get_R(x, y, x+1, y, 4) + \
                                 self.get_R(x, y, x-1, y, 4) + \
                                 self.get_R(x, y, x, y+1, 4) + \
                                 self.get_R(x, y, x, y-1, 4))*dt/s
            return R_4_summary_value
        elif mode == 8:
            R_8_summary_value = (math.pow(self.get_R(x, y, x+1, y+1, 8), 2) + \
                                 math.pow(self.get_R(x, y, x+1, y-1, 8), 2) + \
                                 math.pow(self.get_R(x, y, x-1, y+1, 8), 2) + \
                                 math.pow(self.get_R(x, y, x-1, y-1, 8), 2)) * dt \
                                 / (2*math.pow(s,2))
            return R_8_summary_value
        else:
            print("get_R_summary: Worng mode! please check the input")
    
    
    def check_state(self, x, y):
        state = self.grid[x][y].state
        if state>0:
            print(str(self.grid[x][y].pos) + "report state " + str(state))
        # if state>=0 and state <1:
        #     return
        # elif state>1 and state<=2:
        #     return
        return
    
    

In [24]:
# build model  16s?
model = ABMModel(dem_array)

In [25]:
model.get_agent_counts()

0


In [26]:
model.set_fire(1000,1000)

Well done, you set up a fire at (1000, 1000)


In [27]:
# for i in range(2):
#     model.step()

In [28]:
dem_array.shape

(1886, 1935)

In [29]:
# show the dem world(Just plot the DEM using plt)
# plt.imshow(dem_array, cmap='gray', origin='lower')
# plt.colorbar(label='Elevation')
# plt.title('DEM Elevation')
# plt.show()